<a href="https://colab.research.google.com/github/LC1332/Needy-Haruhi/blob/main/notebook/Needy%E5%90%8E%E5%8F%B0%E7%89%88%E6%9C%AC%E6%A1%86%E6%9E%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!rm -rf /content/Needy-Haruhi
!git clone https://github.com/LC1332/Needy-Haruhi.git


/content
Cloning into 'Needy-Haruhi'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 45 (delta 15), reused 22 (delta 4), pack-reused 0
Receiving objects: 100% (45/45), 26.75 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import sys
sys.path.append('/content/Needy-Haruhi/src')


# Agent系统

agent已经被移动到 src/Agent.py

In [3]:
from Agent import Agent

if __name__ == "__main__":
    # 示例用法
    agent = Agent()
    print(agent["Stress"])  # 输出 0
    agent["Stress"] += 1
    print(agent["Stress"])  # 输出 1
    agent.apply_attribute_change({"Darkness": -1, "Stress": 1})
    print(agent["Darkness"])  # 输出 -1
    print(agent["Stress"])  # 输出 2
    agent.apply_attribute_change({"Nonexistent": 5})  # 输出 Warning: Nonexistent not in attributes, skipping

    condition = ('Stress', 0, 19)

    print( agent.in_condition( condition ) )

0
1
-1
2
True


## Attribute String解析



In [4]:
from util import parse_attribute_string

# Test cases
print(parse_attribute_string("Stress: -1.0, Affection: +0.5"))  # Output: {'Stress': -1.0, 'Affection': 0.5}
print(parse_attribute_string("Affection: +4.0, Stress: -2.0, Darkness: -1.0"))  # Output: {'Affection': 4.0, 'Stress': -2.0, 'Darkness': -1.0}
print(parse_attribute_string("Affection: +2.0, Stress: -1.0, Darkness: ?"))  # Output: {'Affection': 2.0, 'Stress': -1.0, 'Darkness': 0}
print(parse_attribute_string("Stress: -1.0"))  # Output: {'Stress': -1.0}


{'Stress': -1.0, 'Affection': 0.5}
{'Stress': -2.0, 'Affection': 4.0, 'Darkness': -1.0}
{'Stress': -1.0, 'Affection': 2.0, 'Darkness': 0}
{'Stress': -1.0}


## Condition字符串解析 parsing_condition_string

In [5]:
from util import parsing_condition_string

# 测试例子
example_inputs = [
    "Random Noon Event: Darkness 0-39",
    "Random Noon Event: Stress 0-19",
    "Random Noon Event: Affection 61+",
    "Random Noon Event: No Attribute"
]

for example_input in example_inputs:
    print(f"example_input:\n{example_input}\nexample_output\n{parsing_condition_string(example_input)}\n")


example_input:
Random Noon Event: Darkness 0-39
example_output
('Darkness', 0, 39)

example_input:
Random Noon Event: Stress 0-19
example_output
('Stress', 0, 19)

example_input:
Random Noon Event: Affection 61+
example_output
('Affection', 61, 100)

example_input:
Random Noon Event: No Attribute
example_output
None



# DialogueEvent

移动DialogueEvent 到src/DialogueEvent.py

In [6]:
from DialogueEvent import DialogueEvent


example_json_str = """{"prefix": "糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？", "options": [{"user": "你可以试试唱歌直播呀！", "reply": "糖糖: 哇！唱歌直播是个好主意！我可以把我的可爱音色展现给大家听听！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做一些搞笑的小品，逗大家开心。", "reply": "糖糖: 哈哈哈，小品确实挺有趣的！我可以挑战一些搞笑角色，给大家带来欢乐！谢谢你的建议！", "attribute_change": "Stress: -1.0"}, {"user": "你可以尝试做游戏直播，和观众一起玩游戏。", "reply": "糖糖: 游戏直播也不错！我可以和观众一起玩游戏，互动更加有趣！谢谢你的建议！", "attribute_change": "Stress: -1.0"}]}"""

# 通过给定的json字符串初始化DialogueEvent实例
event = DialogueEvent(example_json_str)

# 通过类似字典的方式访问数据
# print(event["options"])  # 打印options字段的内容

print(event.transfer_output(1) )

print(event.get_most_neutral())

print(event.most_neutral_output())



糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？
男主: 你可以尝试做一些搞笑的小品，逗大家开心。
糖糖: 哈哈哈，小品确实挺有趣的！我可以挑战一些搞笑角色，给大家带来欢乐！谢谢你的建议！

0
糖糖: 嘿嘿，最近我在想要不要改变直播风格，你觉得我应该怎么做呀？
男主: 你可以试试唱歌直播呀！
糖糖: 哇！唱歌直播是个好主意！我可以把我的可爱音色展现给大家听听！谢谢你的建议！



批量载入，形成events

In [7]:
file_name =  "/content/Needy-Haruhi/data/original_story_23.jsonl"

import json

events = []

with open(file_name, encoding='utf-8') as f:
    for line in f:
        event = DialogueEvent( line )
        events.append( event )

print(events[0].most_neutral_output())

print(events[0]["id"])

print(events[0]["category"])

print(events[0]["name"])

print(events[0]["condition"])

print(events[0].get_emoji(0))

糖糖: 我正在想下次搞什么企划呢～阿P帮帮我 出出主意
男主: 比如一直打游戏到通关？
糖糖:那就这么办吧（超听话）

Event_Advice_JINE001
Random Noon Event: Darkness 0-39
Event_Advice_JINE001
('Darkness', 0, 39)
📄📄


# Memory

memory我们希望Event和Memory是分离的Event的标准字段如下

- Name， Event的Name，用来后续如果玩家进行游戏修改的话可以根据
- Text， 这个event下完整的对话文本
- Embedding， text的embedding
- Condition， 这个event对应的出现条件
- Emoji， 这个memory的缩写显示emoji

Memory应该可以从Event去默认load一个

In [8]:
example_memory_json = {
    "Name": "EventName",
    "Text": "Sample Text",
    "Embedding": [0,0,0],
    "Condition": "",
    "Emoji": "😓🤯"
}

Memory会包含下面几个字段

example_memory_json = {
    "Name": "EventName",
    "Text": "Sample Text",
    "Embedding": [0,0,0],
    "Condition": "",
    "Emoji": "😓🤯"
}

请为我创建一个Memory类

这个memory类可以通过Memory(json_str)来载入

同时这个类也有和DIalogueEvent类似的get和setitem的功能

我感觉memory直接使用一个MemoryPool的类来进行管理就可以

MemoryPool会有几个方法

- load_from_events 这个是不带Embedding的，会自动抽取Embedding
- save， 保存成一个jsonl格式，其中embedding是base64压缩的
- load， 从jsonl读取
-

In [9]:
# Memory 类不再使用

# import json

# class Memory:
#     def __init__(self, json_str=None):
#         if json_str:
#             try:
#                 self.data = json.loads(json_str)
#             except json.JSONDecodeError:
#                 print("输入的字符串不是有效的JSON格式。")
#                 self.data = {}
#         else:
#             self.data = {}

#     def load_from_event( event ):
#         pass

#     def __getitem__(self, key):
#         return self.data.get(key, None)

#     def __setitem__(self, key, value):
#         self.data[key] = value

#     def __repr__(self):
#         return str(self.data)


# example_memory_json = {
#     "Name": "EventName",
#     "Text": "Sample Text",
#     "Embedding": [0, 0, 0],
#     "Condition": "",
#     "Emoji": "😓🤯"
# }

# # 通过给定的json字符串初始化Memory实例
# memory = Memory(json.dumps(example_memory_json))

# # 通过类似字典的方式访问数据
# print(memory["Name"])  # 打印Name字段的内容
# print(memory["Emoji"])  # 打印Emoji字段的内容


EventName
😓🤯


In [10]:
!pip install -q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00


In [11]:
from transformers import AutoModel, AutoTokenizer
import torch

_bge_model_zh = None
_bge_tokenizer_zh = None

def get_bge_embeddings_zh( sentences ):
    # unsafe ensure batch size by yourself

    global _bge_model_zh
    global _bge_tokenizer_zh

    if _bge_model_zh is None:
        from transformers import AutoTokenizer, AutoModel
        _bge_tokenizer_zh = AutoTokenizer.from_pretrained('BAAI/bge-small-zh-v1.5')
        _bge_model_zh = AutoModel.from_pretrained('BAAI/bge-small-zh-v1.5')

    _bge_model_zh.eval()

    # Tokenize sentences
    encoded_input = _bge_tokenizer_zh(sentences, padding=True, truncation=True, return_tensors='pt', max_length = 512)

    # Compute token embeddings
    with torch.no_grad():
        model_output = _bge_model_zh(**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]
    # normalize embeddings
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.cpu().tolist()

def get_bge_embedding_zh( text_or_texts ):
    if isinstance(text_or_texts, str):
        return get_bge_embeddings_zh([text_or_texts])[0]
    else:
        return get_bge_embeddings_zh(text_or_texts)



In [12]:
result = get_bge_embedding_zh("你好")
print( result )

[0.05038850009441376, 0.043097615242004395, 0.08578279614448547, 0.006132005248218775, 0.044682249426841736, -0.03542076051235199, 0.025161979719996452, 0.028210807591676712, 0.024695759639143944, 0.027889616787433624, 0.10825352370738983, -0.3072983920574188, -0.0445849634706974, -0.0006574562285095453, -0.022935776039958, 0.009746355935931206, -0.013806928880512714, 0.008809655904769897, -0.06698809564113617, -0.030989129096269608, 0.009912646375596523, -0.041775815188884735, 0.008844621479511261, -0.007083534728735685, 0.040530014783144, 0.027941545471549034, 0.022158024832606316, -0.03800597041845322, -0.05458453670144081, -0.025864573195576668, -0.003181352047249675, -0.08981309831142426, -0.005509674083441496, 0.00936888623982668, -0.034590937197208405, -0.019204378128051758, -0.05919954180717468, 0.013990611769258976, -0.016512390226125717, 0.0300595685839653, -0.01522169541567564, 0.007623304147273302, -0.02415050007402897, -0.006122570484876633, 0.006361173931509256, -0.046605

TODO:

- [x] 调一下bge_small
- [x] 写一下load_from_events
- [x] 写一下save和load

In [15]:
from tqdm import tqdm

class MemoryPool:
    def __init__(self):
        self.memories = {}

        self.set_embedding( get_bge_embedding_zh )

    def set_embedding( self, embedding ):
        self.embedding = embedding

    def load_from_events( self, events ):
        for event in tqdm( events ):
            text = event.most_neutral_output()
            embedding = self.embedding( text )
            emoji = event["prefix_emoji"]
            memory_attribute = "" # TODO
            name = event["name"]

            memory = {
                "name": name,
                "text": text,
                "embedding": embedding,
                "memory_attribute": memory_attribute,
                "emoji": emoji # TODO
            }

            self.memories[ name ] = memory

# 我希望为这个类进一步实现save和load函数，save函数可以将memories中的每一个value对应的dict，存储到一个jsonl中，load函数可以读取回来。注意编码都要使用utf-8, ensure_ascii = False

    def save(self, file_name):
        """
        Save the memories dictionary to a jsonl file.
        """
        with open(file_name, 'w', encoding='utf-8') as file:
            for memory in tqdm(self.memories.values()):
                json_record = json.dumps(memory, ensure_ascii=False)
                file.write(json_record + '\n')

    def load(self, file_name):
        """
        Load memories from a jsonl file into the memories dictionary.
        """
        with open(file_name, 'r', encoding='utf-8') as file:
            for line in tqdm(file):
                memory = json.loads(line.strip())
                self.memories[memory['name']] = memory


    def change_memory( self, memory_name , new_text ):
        pass

    def retrieve( agent, query_text ):
        pass

memory_pool = MemoryPool()
memory_pool.load_from_events( events )

memory_pool.save("memory_pool.jsonl")
memory_pool.load("memory_pool.jsonl")



100%|██████████| 23/23 [00:00<00:00, 1027.41it/s]
23it [00:00, 1332.85it/s]


In [17]:
print(memory_pool.memories[list(memory_pool.memories.keys())[0]])





{'name': 'Event_Advice_JINE001', 'text': '糖糖: 我正在想下次搞什么企划呢～阿P帮帮我 出出主意\n男主: 比如一直打游戏到通关？\n糖糖:那就这么办吧（超听话）\n', 'embedding': [0.002292723162099719, 0.02949404902756214, 0.009405561722815037, 0.0440058708190918, -0.014423998072743416, -0.024256421253085136, -0.047747764736413956, 0.05321034789085388, -0.06738205999135971, 0.0327933095395565, 0.09224429726600647, -0.3719632029533386, -0.11805228888988495, -0.05472917854785919, -0.004580599255859852, 0.006189336068928242, 0.057040274143218994, 0.008912482298910618, -0.01382911205291748, -0.03249835595488548, -0.018301956355571747, -0.03298800811171532, 0.028939763084053993, 0.05641625076532364, 0.11260654032230377, -0.012770766392350197, 0.04890493303537369, 0.02928582951426506, -0.041359033435583115, -0.023268986493349075, -0.024331429973244667, -0.08133924007415771, 0.0644974410533905, 0.05515966936945915, 0.016765573993325233, 0.0006056315614841878, -0.02337242290377617, -0.03779278323054314, -0.013056544587016106, 0.041804805397987366, -0.

TODO:

把之前读入的events也都默认转入到Memory

做一下embedding的函数

我希望使用python实现一个简单的文字对话游戏

我希望先实现一个GameMaster类

这个类会不断的和用户对话

GameMaster类会有三个状态，

在Menu状态下，GameMaster会询问玩家是

```
1. 随机一个事件
2. 自由聊天
```

当玩家选择1的时候，GameMaster的交互会交给 EventMaster

当玩家选择2的时候，GameMaster的交互会交给 ChatMaster

当玩家在EventMaster的时候，会经历一次选择，之后就会退出

在ChatMaster的时候，如果玩家输入quit，则会退出，不然则会继续聊天。

请为我编写合适的框架，如果有一些具体的函数，可以先用pass实现。

In [ ]:
class EventMaster:
    def run(self, agent ):
        print("事件发生了！")
        # 这里可以添加事件相关的逻辑
        input("按任意键继续...")


ChatMaster实际上需要

根据agent的属性 先去filter一遍事件

然后从剩余事件中，找到和当前text最接近的k个embedding，放入ChatHaruhi架构中

In [ ]:



class ChatMaster:

    def __init__(self):
        self.top_K = 7

        self.memory_pool = MemoryPool()


    def run(self, agent):
        while True:
            user_input = input("聊天：")
            user_input = user_input.strip()

            if "quit" in user_input or "Quit" in user_input:
                break

            # 首先根据当前的agent，判断每个事件的条件是不是吻合，如果吻合则加入搜索

            #


In [ ]:
class GameMaster:
    def __init__(self, agent = None):
        self.state = "Menu"
        if agent is None:
            self.agent = Agent()


    def run(self):
        while True:
            if self.state == "Menu":
                self.menu()
            elif self.state == "EventMaster":
                self.event_master()
                self.state = "Menu"
            elif self.state == "ChatMaster":
                self.chat_master()
            elif self.state == "Quit":
                break

    def menu(self):
        print("1. 随机一个事件")
        print("2. 自由聊天")
        # (opt) 结局系统
        # 放动画
        # 后台修改attribute
        print("或者输入Quit退出")
        choice = input("请选择一个选项: ")
        if choice == "1":
            self.state = "EventMaster"
        elif choice == "2":
            self.state = "ChatMaster"
        elif "quit" in choice or "Quit" in choice or "QUIT" in choice:
            self.state = "Quit"
        else:
            print("无效的选项，请重新选择")

    def event_master(self):
        event_master = EventMaster()
        event_master.run(self.agent)
        self.state = "Menu"

    def chat_master(self):
        chat_master = ChatMaster()
        chat_master.run(self.agent)
        self.state = "Menu"




In [ ]:
game_master = GameMaster()
game_master.run()

1. 随机一个事件
2. 自由聊天
请选择一个选项: 1
事件发生了！
按任意键继续...
1. 随机一个事件
2. 自由聊天
请选择一个选项: 2
聊天：

False
聊天：Quit
Quit
False
聊天：Quit
Quit
False
聊天：quit
quit
True
1. 随机一个事件
2. 自由聊天
请选择一个选项: quit
